In [1]:
import pandas as pd
import numpy as np
import copy
import datetime
import multiprocessing as mp
from matplotlib import pyplot as plt

In [2]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
arquivos = ['matomo_completo_30062020.csv','matomo_completo_24-26062020.csv','matomo_completo_22-23062020.csv','matomo_completo_17-19062020.csv','matomo_completo_15-16062020.csv','matomo_completo_10-12062020.csv','matomo_completo_08-09062020.csv','matomo_completo_04-05062020.csv','matomo_completo_02062020.csv']
arquivos = ['matomo_completo_30062020.csv']
#arquivos.append('matomo_completo_01-03062020.csv')
#arquivos.append('matomo_completo_04-05062020.csv')
##arquivos.append('matomo_completo_08-09062020.csv')
#arquivos.append('matomo_completo_10-12062020.csv')
#arquivos.append('matomo_completo_15-16062020.csv')
#arquivos.append('matomo_completo_17-19062020.csv')
#arquivos.append('matomo_completo_22-23062020.csv')
#arquivos.append('matomo_completo_24-26062020.csv')
#arquivos.append('matomo_completo_29062020.csv')


In [4]:
arquivos

['matomo_completo_30062020.csv']

In [8]:
df = pd.DataFrame([],columns=['CD_VIS', 'DT_ACAO', 'HR_LCL_VIST', 'TX_IP_VIS', 'CD_UOR', 'NM_UOR_RDZ',
       'HR_ACAO', 'TEMPO_RESPOSTA', 'TX_IDFC_USU', 'UF', 'NM_TIP_ACAO',
       'HORA_TRS', 'DIA_MES', 'DIA_PICO', 'DIA_UTIL', 'CONTEXTO', 'APP',
       'HORA_LOCAL', 'DT_ACAO_DATETIME', 'HORA_COMERCIAL',
       'TEMPO_RESPOSTA_SEGUNDOS', 'TIMEOUT'])
for arq in arquivos:
    df_temp = pd.read_csv('../data/'+arq,sep=';',low_memory=False)#.sample(2000,random_state=1024)
    
    df = pd.concat([df,df_temp],sort=False)

df_temp =[]
    

In [9]:
len(df)

2000

In [7]:
df.columns

Index(['CD_VIS', 'DT_ACAO', 'HR_LCL_VIST', 'TX_IP_VIS', 'CD_UOR', 'NM_UOR_RDZ',
       'HR_ACAO', 'TEMPO_RESPOSTA', 'TX_IDFC_USU', 'UF', 'NM_TIP_ACAO',
       'HORA_TRS', 'DIA_MES', 'DIA_PICO', 'DIA_UTIL', 'CONTEXTO', 'APP',
       'HORA_LOCAL', 'DT_ACAO_DATETIME', 'HORA_COMERCIAL',
       'TEMPO_RESPOSTA_SEGUNDOS', 'TIMEOUT'],
      dtype='object')

In [7]:
df.head().T

,0,1,2,3,4
CD_VIS,200000000820429078,200000000820429078,200000000820429078,200000000820429078,200000000820429078
DT_ACAO,2020-06-30,2020-06-30,2020-06-30,2020-06-30,2020-06-30
HR_LCL_VIST,09:19:37,09:19:37,09:19:37,09:19:37,09:19:37
TX_IP_VIS,10.25.91.157,10.25.91.157,10.25.91.157,10.25.91.157,10.25.91.157
CD_UOR,2442,2442,2442,2442,2442
NM_UOR_RDZ,SAO MATEUS,SAO MATEUS,SAO MATEUS,SAO MATEUS,SAO MATEUS
HR_ACAO,12:22:49,12:50:42,12:31:01,12:49:05,13:20:54
TEMPO_RESPOSTA,922086016.00,105460000.00,604361984.00,443278016.00,903910016.00
TX_IDFC_USU,F2759809,F2759809,F2759809,F2759809,F2759809
UF,ES,ES,ES,ES,ES


In [9]:
df.dtypes

CD_VIS                      object
DT_ACAO                     object
HR_LCL_VIST                 object
TX_IP_VIS                   object
CD_UOR                      object
NM_UOR_RDZ                  object
HR_ACAO                     object
TEMPO_RESPOSTA             float64
TX_IDFC_USU                 object
UF                          object
NM_TIP_ACAO                 object
HORA_TRS                    object
DIA_MES                     object
DIA_PICO                    object
DIA_UTIL                    object
CONTEXTO                    object
APP                         object
HORA_LOCAL                  object
DT_ACAO_DATETIME            object
HORA_COMERCIAL              object
TEMPO_RESPOSTA_SEGUNDOS     object
TIMEOUT                     object
dtype: object

In [10]:
df.isnull().sum()

CD_VIS                             0
DT_ACAO                            0
HR_LCL_VIST                        0
TX_IP_VIS                          0
CD_UOR                             0
NM_UOR_RDZ                         0
HR_ACAO                            0
TEMPO_RESPOSTA                     0
TX_IDFC_USU                  6680026
UF                                 0
NM_TIP_ACAO                        0
HORA_TRS                   111139712
DIA_MES                    111139712
DIA_PICO                   111139712
DIA_UTIL                   111139712
CONTEXTO                   111139712
APP                        111139712
HORA_LOCAL                 111139712
DT_ACAO_DATETIME           111139712
HORA_COMERCIAL             111139712
TEMPO_RESPOSTA_SEGUNDOS    111139712
TIMEOUT                    111139712
dtype: int64

In [11]:
df.dropna(subset=['UF', 'NM_TIP_ACAO'],inplace=True)

In [12]:
cores = mp.cpu_count()  #Number of CPU cores on your system
partitions = cores 
print(partitions)

24


In [13]:
def parallelize(data, func):
    data_split = np.array_split(data, partitions)
    pool = mp.Pool(cores)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

In [14]:
def getContexto(x):
    if type(x) == float:
        return 'Sem Contexto'
    
    pos = x.find('estatico')
    
    if  pos == -1:
        posInicial = 33
        if x[32:33] == ":":
           posInicial = x.find('/')
        
    else:
        posInicial = pos + 9
        
    ind = x[posInicial:len(x)]
    
    posFinal = ind.find('/')
    contexto = ind[0:posFinal]
    
    if contexto == "":
        contexto = 'Sem Contexto'
    
    return contexto
    

In [15]:
def getApp(x):
    
    if type(x) == float:
        return 'Sem App'
        
    pos = x.find('#')
        
    if pos != -1:
        posFinal = pos
        
    pos = x.find('?')

    if pos != -1:
        posFinal = pos
    else:
        posFinal = len(x)
   
  
    posInicial = getPosInicioApp(x[0:posFinal])    
    
    app = x[posInicial:len(x)]
    
    app = getRetiraSujeira(app)
    
    return app
  

In [16]:
def getRetiraSujeira(x):
    
    posicao = x.find("?")
    app = x
    
    if posicao != -1:
        app = x[0:posicao]
    
    return app    

In [17]:
def getPosInicioApp(x):
    pos = 0
    for i in range(len(x),0,-1):
        ch = x[i-1]
        
        if ch == '/':
            pos = i
            break
            
    return pos

In [18]:
def getHoraLocal(x):
    hora = int(x.HORA_TRS) - 3 
    if hora == -1:
        hora = 23
    elif hora == -2:
        hora = 22
    elif hora == -3:
        hora = 21 
    
    if x.UF in ['AM','RR','RO']:
        hora = int(x.HORA_TRS) - 4
        
        if hora == -1:
            hora = 23
        elif hora == -2:
            hora = 22
        elif hora == -3:
            hora = 21 
        elif hora == -4:
            hora = 20
            
    if x.UF in 'AC':
        hora = int(x.HORA_TRS) - 5
        
        if hora == -1:
            hora = 23
        elif hora == -2:
            hora = 22
        elif hora == -3:
            hora = 21 
        elif hora == -4:
            hora = 20
        elif hora == -5:
            hora = 19
          
    return hora
    

In [19]:
def setDiaUtil(x):
    x['DIA_UTIL'] = 1
    x.loc[x['DT_ACAO'] == '2020-06-06','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-07','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-11','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-13','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-14','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-20','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-21','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-27','DIA_UTIL'] = 0
    x.loc[x['DT_ACAO'] == '2020-06-28','DIA_UTIL'] = 0
    return x

In [20]:
def getDiaUtil(x):
    return datetime.datetime.x.weekday()


In [21]:
def processaArquivo(data): 
    data['HORA_TRS'] = data['HR_ACAO'].str[0:2]  
    data['DIA_MES'] = data['DT_ACAO'].str[8:10]
    
    data['DIA_PICO'] = data.DIA_MES.apply(lambda x: 1 if x in ['30','31','01','02','03','04','05','06','07','08','09','10'] else 0)
   
    data['DIA_UTIL'] = 1
    data.loc[data['DT_ACAO'] == '2020-06-06','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-07','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-11','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-13','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-14','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-20','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-21','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-27','DIA_UTIL'] = 0
    data.loc[data['DT_ACAO'] == '2020-06-28','DIA_UTIL'] = 0
    
    data['NM_TIP_ACAO'] = data.NM_TIP_ACAO.apply(lambda x: x.strip())
    
    data['CONTEXTO'] = data.NM_TIP_ACAO.apply(lambda x: getContexto(x))
                                                 
    data['APP'] = data.NM_TIP_ACAO.apply(lambda x: getApp(x))   
    
    data['HORA_LOCAL'] = data.apply(lambda x: getHoraLocal(x),axis=1)
    data['DT_ACAO_DATETIME'] = pd.to_datetime(data['DT_ACAO'])
    
    data['HORA_COMERCIAL'] = 0
    data.loc[(data['HORA_TRS'] >= '08') | (data['HORA_TRS'] >= '18'),'HORA_COMERCIAL'] = 1
    
    data['TEMPO_RESPOSTA_SEGUNDOS'] = data['TEMPO_RESPOSTA']/1000

    data['TIMEOUT'] = 0
    data.loc[data['TEMPO_RESPOSTA_SEGUNDOS'] > 30,'TIMEOUT'] = 1      
    
     
    return data

In [22]:
df = parallelize(df,processaArquivo)

In [23]:
media_temp = df.groupby('APP')['TEMPO_RESPOSTA_SEGUNDOS'].agg({'TEMPO_RESPOSTA_SEGUNDOS':'mean'},index=False).reset_index().rename(columns={"TEMPO_RESPOSTA_SEGUNDOS":"Média"})

/Users/adrianocastanho/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


In [24]:
media_temp.head(20).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
APP,,Call - Opção de compr,01.06.2020,02.06.2020,03.06.2020,04.06.2020,05.06.2020,07.05.2020,08.06.2020,09.06.2020,1,10.06.2020,11.06.2020,12.06.2020,15.06.2020,16.06.2020,17.06.2020,18.06.2020,19.06.2020,2
Média,0.01,0.47,0.77,0.82,0.78,2.16,0.73,0.55,0.65,0.55,0.64,0.60,0.50,0.60,0.52,0.72,0.54,0.65,0.64,5.20


In [25]:
media_temp.to_csv('../data/media_apps.csv',sep=';',index=False)

In [26]:
teste = pd.qcut(df['TEMPO_RESPOSTA_SEGUNDOS'], q=8, duplicates='drop', precision=2)

In [27]:
print(teste)

0          (3.3, 937273.98]
1          (3.3, 937273.98]
2          (3.3, 937273.98]
3          (3.3, 937273.98]
4          (3.3, 937273.98]
                 ...       
5561297         (2.09, 3.3]
5561298        (1.42, 2.09]
5561299        (0.59, 0.92]
5561300        (-0.01, 0.1]
5561301        (-0.01, 0.1]
Name: TEMPO_RESPOSTA_SEGUNDOS, Length: 111139712, dtype: category
Categories (8, interval[float64]): [(-0.01, 0.1] < (0.1, 0.39] < (0.39, 0.59] < (0.59, 0.92] < (0.92, 1.42] < (1.42, 2.09] < (2.09, 3.3] < (3.3, 937273.98]]


In [38]:
df['CLASSE_APP'] = pd.cut(df['TEMPO_RESPOSTA_SEGUNDOS'], bins=[-0.01,0.1,0.39, 0.59, 0.92, 1.42, 2.09, 3.3, 30.0,937274])

In [39]:
df.CLASSE_APP.cat.rename_categories({pd.Interval(-0.01, 0.1) : 'Super Baixo (até 0.1 seg)',
                                                      pd.Interval(0.1, 0.39) : 'Muito Baixo (0.1 - 0.39 seg)',
                                                      pd.Interval(0.39, 0.59) : 'Baixo (0.39 - 0.59 seg)',
                                                      pd.Interval(0.59,0.92) : 'Médio (0.59 - 0.92 seg)',
                                                      pd.Interval(0.92,1.42) : 'Médio (0.91 - 1.42 seg)',
                                                      pd.Interval(1.42, 2.09) : 'Alto (1.42 - 2.09 seg)', 
                                                      pd.Interval(2.09, 3.3) : 'Muito Alto (2.09 - 3.3 seg)',
                                                      pd.Interval(3.3, 30) : 'Super Alto (3.24 - 30 seg)',
                                                      pd.Interval(30.0,937274): 'TimeOut'}, 
                                                      inplace=True)

In [40]:
df.head().T

,0,1,2,3,4
CD_VIS,200000000820429078,200000000820429078,200000000820429078,200000000820429078,200000000820429078
DT_ACAO,2020-06-30,2020-06-30,2020-06-30,2020-06-30,2020-06-30
HR_LCL_VIST,09:19:37,09:19:37,09:19:37,09:19:37,09:19:37
TX_IP_VIS,10.25.91.157,10.25.91.157,10.25.91.157,10.25.91.157,10.25.91.157
CD_UOR,2442,2442,2442,2442,2442
NM_UOR_RDZ,SAO MATEUS,SAO MATEUS,SAO MATEUS,SAO MATEUS,SAO MATEUS
HR_ACAO,12:22:49,12:50:42,12:31:01,12:49:05,13:20:54
TEMPO_RESPOSTA,922086016.00,105460000.00,604361984.00,443278016.00,903910016.00
TX_IDFC_USU,F2759809,F2759809,F2759809,F2759809,F2759809
UF,ES,ES,ES,ES,ES


In [41]:
df.isnull().sum()

CD_VIS                           0
DT_ACAO                          0
HR_LCL_VIST                      0
TX_IP_VIS                        0
CD_UOR                           0
NM_UOR_RDZ                       0
HR_ACAO                          0
TEMPO_RESPOSTA                   0
TX_IDFC_USU                6680026
UF                               0
HORA_TRS                         0
DIA_MES                          0
DIA_PICO                         0
DIA_UTIL                         0
CONTEXTO                         0
APP                              0
HORA_LOCAL                       0
DT_ACAO_DATETIME                 0
HORA_COMERCIAL                   0
TEMPO_RESPOSTA_SEGUNDOS          0
TIMEOUT                          0
CLASSE_APP                       0
dtype: int64

In [42]:
#temp = df.groupby(['CONTEXTO','APP','NM_TIP_ACAO']).agg({'TEMPO_RESPOSTA_SEGUNDOS':'count'},index=False).reset_index().rename(columns={"TEMPO_RESPOSTA_SEGUNDOS": "Qtde"})
#temp = df.groupby(['CONTEXTO','APP','NM_TIP_ACAO']).agg({'TEMPO_RESPOSTA_SEGUNDOS':'count'},index=False).reset_index().rename(columns={"TEMPO_RESPOSTA_SEGUNDOS": "Qtde"})
#temp.to_csv('../data/contextos_app.csv',index=False,sep=';')
#temp

In [43]:
df.TEMPO_RESPOSTA_SEGUNDOS.describe()

count   111139712.00
mean         1352.46
std         31534.12
min             0.00
25%             0.39
50%             0.92
75%             2.09
max        937273.98
Name: TEMPO_RESPOSTA_SEGUNDOS, dtype: float64

In [44]:
df.isnull().sum()

CD_VIS                           0
DT_ACAO                          0
HR_LCL_VIST                      0
TX_IP_VIS                        0
CD_UOR                           0
NM_UOR_RDZ                       0
HR_ACAO                          0
TEMPO_RESPOSTA                   0
TX_IDFC_USU                6680026
UF                               0
HORA_TRS                         0
DIA_MES                          0
DIA_PICO                         0
DIA_UTIL                         0
CONTEXTO                         0
APP                              0
HORA_LOCAL                       0
DT_ACAO_DATETIME                 0
HORA_COMERCIAL                   0
TEMPO_RESPOSTA_SEGUNDOS          0
TIMEOUT                          0
CLASSE_APP                       0
dtype: int64

In [45]:
df.drop(columns=["NM_TIP_ACAO"],inplace=True)

KeyError: "['NM_TIP_ACAO'] not found in axis"

In [46]:
df.to_csv('../data/base_plataforma_completo_final.csv',sep=';',index=False)